In [1]:
import pandas as pd
import numpy as np
from surprise import NMF
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

from read_and_split_data import split_data, filter_animes_without_grade

In [2]:
def get_dataset():
    anime = pd.read_parquet("../datasets/anime.parquet")
    anime = anime[["anime_id", "type"]]
    users = pd.read_parquet("../datasets/users.parquet")
    base_df = users.merge(anime, on="anime_id", how="left")
    return base_df

base_df = get_dataset()
data = split_data(base_df)

In [3]:
kf = KFold(n_splits=5)

In [4]:
def predict_and_error(data, algo, path):
    fold = 0
    predictions_list = []
    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        predictions_list.append(predictions)
        # Mean Squared Error
        accuracy.rmse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mae(predictions, verbose=True)
        # Fraction of Concordant Pairs
        accuracy.fcp(predictions, verbose=True)
        base_df = pd.DataFrame(columns=["user", "anime", "actual", "est", "details"], data=predictions)
        base_df.to_csv(f"{path}_{fold}.csv")
        base_df.to_parquet(f"{path}_{fold}.parquet")
        fold += 1
    return predictions

In [5]:
algo_normal_predictor = NMF()

In [6]:
predictions = predict_and_error(data, algo_normal_predictor, path="../predictions/nmf/anime_type_tv/no_filter/nmf")

RMSE: 6.0857
MSE: 37.0361
MAE:  5.5873
FCP:  0.2530
RMSE: 6.2663
MSE: 39.2661
MAE:  5.7852
FCP:  0.2052
RMSE: 6.3763
MSE: 40.6576
MAE:  5.9230
FCP:  0.1392
RMSE: 6.3921
MSE: 40.8594
MAE:  5.9412
FCP:  0.1500
RMSE: 6.3318
MSE: 40.0916
MAE:  5.8714
FCP:  0.1873


In [7]:
base_df_without_negative = filter_animes_without_grade(base_df)

In [8]:
predictions_without_negative = predict_and_error(data, algo_normal_predictor, path="../predictions/nmf/anime_type_tv/with_filter_remove_negative/nmf")

RMSE: 6.3807
MSE: 40.7128
MAE:  5.9267
FCP:  0.1433
RMSE: 6.0820
MSE: 36.9903
MAE:  5.5762
FCP:  0.2475
RMSE: 6.3196
MSE: 39.9374
MAE:  5.8528
FCP:  0.1489
RMSE: 6.4153
MSE: 41.1566
MAE:  5.9709
FCP:  0.1210
RMSE: 6.1610
MSE: 37.9573
MAE:  5.6682
FCP:  0.2383
